In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

import os

In [2]:
dataDir = "data/"
growth_rate = pd.read_csv(dataDir + "covid_clean.csv", encoding="latin_1")
growth_rate

,fips,day_1,day_2,day_3,day_4,day_5,day_6,day_7,day_8,day_9,...,day_91,day_92,day_93,day_94,day_95,day_96,day_97,day_98,statefips,start_date
0,1001,0.000000,0.000000,0.000000,0.154151,0.000000,0.356675,0.000000,0.182322,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000,2020-03-26
1,1003,0.405465,0.182322,0.054067,0.233615,0.080043,0.109199,0.033902,0.154151,0.108214,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000,2020-03-28
2,1005,0.251314,0.105361,0.000000,0.000000,0.095310,0.167054,0.074108,0.068993,0.182322,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000,2020-04-09
3,1007,0.000000,0.133531,0.117783,0.200671,0.000000,0.167054,0.207639,0.060625,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000,2020-04-05
4,1009,0.405465,0.105361,0.000000,0.000000,0.000000,0.000000,0.095310,0.087011,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000,2020-04-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2487,56033,0.000000,0.287682,0.223144,0.000000,0.000000,0.000000,0.095310,0.087011,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56000,2020-03-28
2488,56037,-0.182322,0.587787,0.000000,0.105361,0.095310,0.167054,0.000000,0.074108,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56000,2020-04-07
2489,56039,0.287682,0.405465,0.154151,0.000000,0.194156,0.302281,0.122602,0.109199,0.098440,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56000,2020-03-25
2490,56041,0.000000,0.000000,0.154151,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56000,2020-04-18


In [3]:
job_shares = pd.read_csv(dataDir + "jobs_shares.csv", encoding="latin_1")
num_fips = []
for fip in job_shares['geofips']:
    num_fips.append(int(fip.strip('"')))
job_shares['geofips'] = num_fips
job_shares = job_shares.rename(columns={"geofips" : 'fips'})

In [4]:
lockdown = pd.read_csv(dataDir + "lockdown_dates.csv")
us_lock = lockdown[lockdown['Country'] == 'United States']
state2fips = pd.read_csv(dataDir + "state_fips.csv").rename(columns={"stname" : "Place"})
state2fips["statefips"] = state2fips[' st'] * 1000
state2fips = state2fips.drop([' st', ' stusps'], axis = 1)

def export2csv(filedir, filename, data):
    if os.path.exists(filedir + filename):
        os.remove(filedir + filename)
        print("File deleted")
    else:
        print("The file does not exist")
    data.to_csv(filedir + filename, index=False, encoding='latin_1')

export2csv(dataDir, "state_fips_clean.csv", state2fips)

File deleted


In [5]:
population = pd.read_csv(dataDir + "dollars_clean.csv", encoding = "latin_1")
population = population[['GeoFIPS', '20']]
population['20'] = population['20'].astype(int)
num_fips = []
for fip in population['GeoFIPS']:
    num_fips.append(int(fip.strip().strip('"')))
population['GeoFIPS'] = num_fips
population = population.rename(columns={"GeoFIPS" : 'fips', "20" : 'population'})

export2csv(dataDir, "population.csv", population)

File deleted


In [6]:
fips_lockdown = pd.merge(us_lock, state2fips, on="Place")[['statefips', 'Start date']]
fips_lockdown = fips_lockdown.rename(columns={"Start date" : "lockdown start"})
fips_lockdown = fips_lockdown.sort_values('statefips')

export2csv(dataDir, "us_lockdown_dates.csv", fips_lockdown)

File deleted


In [7]:
temp = pd.merge(growth_rate, fips_lockdown, on='statefips', how = 'outer')
temp['lockdown_delta'] = (pd.to_datetime(temp['lockdown start']) - pd.to_datetime(temp['start_date'])).dt.days
temp = pd.merge(temp, population, on='fips')
#temp[temp['fips'] == 53033].head(10)#

In [8]:
export = pd.merge(temp, job_shares, on='fips')
drop_cols = ['statefips', 'start_date', 'lockdown start']
for idx, col in enumerate(job_shares.columns):
    if idx < 9 and idx > 0:
        drop_cols.append(col)
export = export.drop(columns = drop_cols)

export2csv(dataDir, "regression_data.csv", export)

File deleted


In [10]:
list(enumerate(export.columns))

[(0, 'fips'),
 (1, 'day_1'),
 (2, 'day_2'),
 (3, 'day_3'),
 (4, 'day_4'),
 (5, 'day_5'),
 (6, 'day_6'),
 (7, 'day_7'),
 (8, 'day_8'),
 (9, 'day_9'),
 (10, 'day_10'),
 (11, 'day_11'),
 (12, 'day_12'),
 (13, 'day_13'),
 (14, 'day_14'),
 (15, 'day_15'),
 (16, 'day_16'),
 (17, 'day_17'),
 (18, 'day_18'),
 (19, 'day_19'),
 (20, 'day_20'),
 (21, 'day_21'),
 (22, 'day_22'),
 (23, 'day_23'),
 (24, 'day_24'),
 (25, 'day_25'),
 (26, 'day_26'),
 (27, 'day_27'),
 (28, 'day_28'),
 (29, 'day_29'),
 (30, 'day_30'),
 (31, 'day_31'),
 (32, 'day_32'),
 (33, 'day_33'),
 (34, 'day_34'),
 (35, 'day_35'),
 (36, 'day_36'),
 (37, 'day_37'),
 (38, 'day_38'),
 (39, 'day_39'),
 (40, 'day_40'),
 (41, 'day_41'),
 (42, 'day_42'),
 (43, 'day_43'),
 (44, 'day_44'),
 (45, 'day_45'),
 (46, 'day_46'),
 (47, 'day_47'),
 (48, 'day_48'),
 (49, 'day_49'),
 (50, 'day_50'),
 (51, 'day_51'),
 (52, 'day_52'),
 (53, 'day_53'),
 (54, 'day_54'),
 (55, 'day_55'),
 (56, 'day_56'),
 (57, 'day_57'),
 (58, 'day_58'),
 (59, 'day_59'),
 